In [2]:
import os
import glob
import numpy as np
import pandas as pd
import music21
from collections import Counter
from scipy.stats import entropy
import warnings

warnings.filterwarnings('ignore')

# 1. Metric Calculation Functions

def calc_note_density(all_notes, score):
    if not all_notes: return 0
    total_duration = score.highestTime
    if total_duration > 0:
        return len(all_notes) / total_duration
    return 0

def calc_avg_duration(all_notes, score):
    durations = [n.duration.quarterLength for n in all_notes]
    if durations:
        return np.mean(durations)
    return 0

def calc_pitch_range(all_notes, score):
    pitches = []
    for n in all_notes:
        if n.isNote:
            pitches.append(n.pitch.ps)
        elif n.isChord:
            pitches.extend([p.ps for p in n.pitches])
    if pitches:
        return max(pitches) - min(pitches)
    return 0

def calc_rhythmic_entropy(all_notes, score):
    durations = [n.duration.quarterLength for n in all_notes]
    if durations:
        counts = Counter(durations)
        probs = [c / len(durations) for c in counts.values()]
        return entropy(probs, base=2)
    return 0

def calc_chord_ratio(all_notes, score):
    if len(all_notes) > 0:
        chord_count = sum(1 for n in all_notes if n.isChord)
        return chord_count / len(all_notes)
    return 0

def calc_pitch_class_entropy(all_notes, score):
    pcs = []
    for n in all_notes:
        if n.isNote:
            pcs.append(n.pitch.pitchClass)
        elif n.isChord:
            pcs.extend([p.pitchClass for p in n.pitches])
    if pcs:
        counts = Counter(pcs)
        probs = [c / len(pcs) for c in counts.values()]
        return entropy(probs, base=2)
    return 0

def calc_melodic_interval_mean(all_notes, score):
    sorted_notes = sorted(all_notes, key=lambda x: x.offset)
    p_seq = []
    for n in sorted_notes:
        if n.isNote: 
            p_seq.append(n.pitch.ps)
        elif n.isChord: 
            if n.pitches:
                p_seq.append(max(p.ps for p in n.pitches))
    
    if len(p_seq) > 1:
        diffs = [abs(p_seq[i] - p_seq[i-1]) for i in range(1, len(p_seq))]
        return np.mean(diffs)
    return 0

METRICS_MAP = {
    'Note Density': calc_note_density,
    'Avg Duration': calc_avg_duration,
    'Pitch Range': calc_pitch_range,
    'Rhythmic Entropy': calc_rhythmic_entropy,
    'Chord Ratio': calc_chord_ratio,
    'Pitch Class Entropy': calc_pitch_class_entropy,
    'Melodic Interval Mean': calc_melodic_interval_mean
}

# 2. Batch Processing and Display

def evaluate_generated_music():
    target_files = glob.glob("*.mid") 
    valid_files = [f for f in target_files if "Baroque" in f or "Romantic" in f]
    
    if not valid_files:
        print("No files named Baroque_x.mid or Romantic_x.mid found.")
        return

    print(f"Found {len(valid_files)} generated files. Starting analysis...\n")
    results = []

    for filepath in valid_files:
        filename = os.path.basename(filepath)
        style = "Baroque" if "Baroque" in filename else "Romantic"
        
        try:
            score = music21.converter.parse(filepath)
            all_notes = score.flat.notes
            
            row = {"Filename": filename, "Style": style}
            for metric_name, func in METRICS_MAP.items():
                row[metric_name] = func(all_notes, score)
            
            results.append(row)
            
        except Exception as e:
            print(f"Error parsing {filename}: {e}")

    if results:
        df = pd.DataFrame(results)
        cols = ['Filename', 'Style'] + list(METRICS_MAP.keys())
        df = df[cols]
        
        print("\n" + "="*80)
        print("Part 1: Individual Sample Report")
        print("="*80)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        print(df.round(2))

        print("\n" + "="*80)
        print("Part 2: Aggregated Comparison (Mean Values)")
        print("="*80)
        
        comparison = df.groupby("Style")[list(METRICS_MAP.keys())].mean().round(2)
        print(comparison)
        
        output_csv = "evaluation_detailed_results.csv"
        df.to_csv(output_csv, index=False)
        print(f"\nDetailed results saved to: {output_csv}")

if __name__ == "__main__":
    evaluate_generated_music()

Found 20 generated files. Starting analysis...


Part 1: Individual Sample Report
           Filename     Style  Note Density  Avg Duration  Pitch Range  Rhythmic Entropy  Chord Ratio  Pitch Class Entropy  Melodic Interval Mean
0     Baroque_1.mid   Baroque          2.71          0.69         31.0              1.19         0.39                 2.91                   4.75
1    Romantic_8.mid  Romantic          1.67          1.02         56.0              2.28         0.47                 3.30                   7.24
2    Romantic_5.mid  Romantic          2.31          0.39         60.0              1.34         0.52                 3.27                   7.62
3    Romantic_1.mid  Romantic          1.17          0.74         73.0              2.80         0.56                 3.12                   3.90
4    Baroque_10.mid   Baroque          2.79          0.80         38.0              2.24         0.34                 3.08                   7.75
5    Romantic_9.mid  Romantic          0.7